In [10]:
from playwright.sync_api import sync_playwright

In [11]:
from playwright.sync_api import sync_playwright
from bs4 import BeautifulSoup, Comment
import pandas as pd


URL = "https://fbref.com/en/players/4b542852/scout/365_m1/Adam-Wharton-Scouting-Report"
TABLE_ID = "scout_full_MF"


def scrape_fbref_scout(url, table_id):
    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        # Load page like a real browser
        page.goto(url, timeout=60000)
        page.wait_for_load_state("networkidle")

        html = page.content()
        browser.close()

    soup = BeautifulSoup(html, "lxml")

    # FBref puts tables inside HTML comments
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))

    for c in comments:
        if table_id in c:
            df = pd.read_html(c)[0]
            return df

    raise ValueError(f"Table {table_id} not found")


# ---- run ----
df = scrape_fbref_scout(URL, TABLE_ID)

# clean columns
df.columns = df.columns.get_level_values(-1)
df = df[df["Statistic"] != "Statistic"]

print(df.head())

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.